In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [0]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_6.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32,
                            'distancia_euclideana_al_origen' : np.float32, 
                            'distancia_minima_comercial' : np.float32,
                            'coseno_maximo_ciudad_pais' : np.float32,
                            'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad' : np.float32,
                            'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra' : np.float32,
                            'antiguedad_sobre_tipodepropiedad_mean_antiguedad' : np.float32,
                            'direccion_cantidad_al_menos_una_mayuscula' : np.float32,
                            'direccion_cantidad_fijos_top_k' : np.float32, 
                            'titulo_cantidad_fijos_top_k' : np.float32,
                            'titulo_palabras_top_k_sobre_total_palabras' : np.float32,
                            'ciudad_distancia_al_origen':  np.float32, 
                            'ciudad_mean_mean_todas' : np.float32,
                            'ciudad_mean_antiguedad_sobre_mean_metrocubiertos' : np.float32
                            }
                    )

In [0]:
train.set_index('id', inplace = True)

In [0]:
TARGET = 'precio'
FEATURES = ['descripcion_longitud_media_de_palabra', 'metrostotales',
       'metros_x_espacio', 'idzona', 'lat', 'titulo_longitud_media_de_palabra',
       'distancia_minima_Oficina_comercial',
       'distancia_minima_Local_en_centro_comercial',
       'distancia_minima_Bodega_comercial', 'metros_x_garages',
       'metroscubiertos', 'metros_x_habitaciones', 'metros_x_banos',
       'distancia_minima_Terreno_comercial',
       'distancia_minima_Local_Comercial',
       'titulo_cantidad_caracteres_en_palabras_importantes',
       'descripcion_cantidad_palabras_top_k',
       'descripcion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_provincia',
       'descripcion_cantidad_stopwords',
       'descripcion_cantidad_signos_puntacion',
       'direccion_longitud_media_de_palabra',
       'titulo_palabras_top_k_sobre_total_palabras',
       'habitaciones_preciopromedio_ciudad', 'antiguedad', 'lng',
       'garages_preciopromedio_ciudad',
       'direccion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_ciudad',
       'banos_preciopromedio_ciudad', 'descripcion_cantidad_prefijos_top_k',
       'precio_x_m2', 'año', 'titulo_cantidad_fijos_top_k',
       'descripcion_cantidad_palabras_importantes',
       'descripcion_cantidad_postfijos_top_k', 'ciudad_mean_mean_todas',
       'ciudad_mean_antiguedad_sobre_mean_metrocubiertos',
       'habitaciones_preciopromedio_metroscubiertos', 'cantidad_espacios',
       'direccion_cantidad_al_menos_una_mayuscula',
       'direccion_cantidad_fijos_top_k', 'ciudad_distancia_al_origen']

In [0]:
X = train[FEATURES]
y = train[TARGET]

In [0]:
K = 3
def objective(hyper_parametros):

    xgb_regressor = XGBRegressor(objective = 'reg:squarederror',**hyper_parametros)
    
    score = cross_val_score(xgb_regressor, X, y, 
                            scoring='neg_mean_absolute_error', cv=K).mean()
    
    print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
    
    return {'loss': -score, 'status': STATUS_OK}

In [0]:
space = {
    "n_estimators": hp.randint("xgb_re__n_estimators", 50) + 600, 
    "learning_rate": hp.uniform("xgb_re__learning_rate", 0.011, 0.029), 
    "gamma": hp.uniform("xgb_re__gamma", 5, 7),
    "lambda": hp.uniform("xgb_re__lambda", 1, 2),
    "max_depth": hp.randint("xgb_re__max_depth", 4) + 18, 
    "min_child_weight": hp.randint("xgb_re__min_child_weight", 50) + 8,
    'colsample_bytree': 0.8,
}

In [0]:
N_ITER = 10
rstate = np.random.RandomState(14) 
t0 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=N_ITER,
            rstate = rstate)
t1 = time.time()
print('Tiempo = {0:.2f} horas'.format((t1 - t0)/3600))

SCORE: -553098.604 params {'colsample_bytree': 0.8, 'gamma': 6.782284556145175, 'lambda': 1.7287324913421842, 'learning_rate': 0.012823791237591541, 'max_depth': 21, 'min_child_weight': 42, 'n_estimators': 609}
SCORE: -549600.958 params {'colsample_bytree': 0.8, 'gamma': 6.13841794293382, 'lambda': 1.363396776977516, 'learning_rate': 0.023943715304410577, 'max_depth': 19, 'min_child_weight': 17, 'n_estimators': 614}
SCORE: -552026.812 params {'colsample_bytree': 0.8, 'gamma': 5.340472361867451, 'lambda': 1.750633857991943, 'learning_rate': 0.014708696075720352, 'max_depth': 18, 'min_child_weight': 27, 'n_estimators': 614}
SCORE: -547578.521 params {'colsample_bytree': 0.8, 'gamma': 6.418961485185021, 'lambda': 1.0558594532067118, 'learning_rate': 0.02810506941893589, 'max_depth': 21, 'min_child_weight': 28, 'n_estimators': 614}
SCORE: -549253.292 params {'colsample_bytree': 0.8, 'gamma': 5.0137587490395905, 'lambda': 1.1360453316012982, 'learning_rate': 0.02261104004959845, 'max_depth'

SCORE: -519440.767 params {'colsample_bytree': 0.8, 'gamma': 4.7506160486034, 'lambda': 1.1183589932730134, 'learning_rate': 0.02611652524078614, 'max_depth': 18, 'min_child_weight': 9, 'n_estimators': 604, 'subsample': 0.8}  

SCORE: -578336.292 params {'gamma': 3.9592400418765896, 'lambda': 3.7750522524494228, 'learning_rate': 0.015465931263479917, 'max_depth': 18, 'min_child_weight': 344, 'n_estimators': 586} 
 

SCORE: -590996.891 params {'colsample_bytree': 0.8, 'gamma': 4.8363434624215715, 'lambda': 1.1296448122275025, 'learning_rate': 0.02314630700688795, 'max_depth': 18, 'min_child_weight': 808, 'n_estimators': 500}  
SCORE: -554942.859 params {'colsample_bytree': 0.8, 'gamma': 4.593052880568019, 'lambda': 1.4716679253486882, 'learning_rate': 0.02981991678123363, 'max_depth': 19, 'min_child_weight': 204, 'n_estimators': 667}  
SCORE: -580034.406 params {'colsample_bytree': 0.8, 'gamma': 3.3973134025851235, 'lambda': 1.4342463882815433, 'learning_rate': 0.021700230132434526, 'max_depth': 17, 'min_child_weight': 571, 'n_estimators': 605}  
SCORE: -544613.219 params {'colsample_bytree': 0.8, 'gamma': 3.303773785585392, 'lambda': 1.8889847537179332, 'learning_rate': 0.028731691846996067, 'max_depth': 18, 'min_child_weight': 40, 'n_estimators': 574}  
SCORE: -573675.797 params {'colsample_bytree': 0.8, 'gamma': 3.507808926409963, 'lambda': 1.806081007945392, 'learning_rate': 0.016115593051986545, 'max_depth': 16, 'min_child_weight': 291, 'n_estimators': 626}  
SCORE: -575812.578 params {'colsample_bytree': 0.8, 'gamma': 3.4298142659340876, 'lambda': 1.2033094509247353, 'learning_rate': 0.029634823983423257, 'max_depth': 17, 'min_child_weight': 587, 'n_estimators': 591}  


SCORE: -590996.891 params {'colsample_bytree': 0.8, 'gamma': 4.8363434624215715, 'lambda': 1.1296448122275025, 'learning_rate': 0.02314630700688795, 'max_depth': 18, 'min_child_weight': 808, 'n_estimators': 500}  
SCORE: -554942.859 params {'colsample_bytree': 0.8, 'gamma': 4.593052880568019, 'lambda': 1.4716679253486882, 'learning_rate': 0.02981991678123363, 'max_depth': 19, 'min_child_weight': 204, 'n_estimators': 667}  
SCORE: -580034.406 params {'colsample_bytree': 0.8, 'gamma': 3.3973134025851235, 'lambda': 1.4342463882815433, 'learning_rate': 0.021700230132434526, 'max_depth': 17, 'min_child_weight': 571, 'n_estimators': 605}  
SCORE: -544613.219 params {'colsample_bytree': 0.8, 'gamma': 3.303773785585392, 'lambda': 1.8889847537179332, 'learning_rate': 0.028731691846996067, 'max_depth': 18, 'min_child_weight': 40, 'n_estimators': 574}  
SCORE: -573675.797 params {'colsample_bytree': 0.8, 'gamma': 3.507808926409963, 'lambda': 1.806081007945392, 'learning_rate': 0.016115593051986545, 'max_depth': 16, 'min_child_weight': 291, 'n_estimators': 626}  
SCORE: -575812.578 params {'colsample_bytree': 0.8, 'gamma': 3.4298142659340876, 'lambda': 1.2033094509247353, 'learning_rate': 0.029634823983423257, 'max_depth': 17, 'min_child_weight': 587, 'n_estimators': 591}  
SCORE: -581495.609 params {'colsample_bytree': 0.8, 'gamma': 3.008899827461131, 'lambda': 1.5159711880171978, 'learning_rate': 0.010571570452367124, 'max_depth': 17, 'min_child_weight': 251, 'n_estimators': 597}  
SCORE: -549725.203 params {'colsample_bytree': 0.8, 'gamma': 4.499847349111102, 'lambda': 1.1294728397775364, 'learning_rate': 0.011335474730383408, 'max_depth': 19, 'min_child_weight': 24, 'n_estimators': 559}  

SCORE: -553098.604 params {'colsample_bytree': 0.8, 'gamma': 6.782284556145175, 'lambda': 1.7287324913421842, 'learning_rate': 0.012823791237591541, 'max_depth': 21, 'min_child_weight': 42, 'n_estimators': 609}
SCORE: -549600.958 params {'colsample_bytree': 0.8, 'gamma': 6.13841794293382, 'lambda': 1.363396776977516, 'learning_rate': 0.023943715304410577, 'max_depth': 19, 'min_child_weight': 17, 'n_estimators': 614}
SCORE: -552026.812 params {'colsample_bytree': 0.8, 'gamma': 5.340472361867451, 'lambda': 1.750633857991943, 'learning_rate': 0.014708696075720352, 'max_depth': 18, 'min_child_weight': 27, 'n_estimators': 614}

SCORE: -553098.604 params {'colsample_bytree': 0.8, 'gamma': 6.782284556145175, 'lambda': 1.7287324913421842, 'learning_rate': 0.012823791237591541, 'max_depth': 21, 'min_child_weight': 42, 'n_estimators': 609}
SCORE: -549600.958 params {'colsample_bytree': 0.8, 'gamma': 6.13841794293382, 'lambda': 1.363396776977516, 'learning_rate': 0.023943715304410577, 'max_depth': 19, 'min_child_weight': 17, 'n_estimators': 614}
SCORE: -552026.812 params {'colsample_bytree': 0.8, 'gamma': 5.340472361867451, 'lambda': 1.750633857991943, 'learning_rate': 0.014708696075720352, 'max_depth': 18, 'min_child_weight': 27, 'n_estimators': 614}
SCORE: -547578.521 params {'colsample_bytree': 0.8, 'gamma': 6.418961485185021, 'lambda': 1.0558594532067118, 'learning_rate': 0.02810506941893589, 'max_depth': 21, 'min_child_weight': 28, 'n_estimators': 614}
SCORE: -549253.292 params {'colsample_bytree': 0.8, 'gamma': 5.0137587490395905, 'lambda': 1.1360453316012982, 'learning_rate': 0.02261104004959845, 'max_depth': 20, 'min_child_weight': 50, 'n_estimators': 634}
SCORE: -547537.375 params {'colsample_bytree': 0.8, 'gamma': 6.809845694575199, 'lambda': 1.5929642745470307, 'learning_rate': 0.025002042818406546, 'max_depth': 21, 'min_child_weight': 26, 'n_estimators': 633}
SCORE: -552058.958 params {'colsample_bytree': 0.8, 'gamma': 6.747949491761628, 'lambda': 1.7459168030014796, 'learning_rate': 0.024078009028246715, 'max_depth': 21, 'min_child_weight': 13, 'n_estimators': 606}
SCORE: -552472.562 params {'colsample_bytree': 0.8, 'gamma': 5.632946149962807, 'lambda': 1.1275490848575256, 'learning_rate': 0.01829268622400661, 'max_depth': 20, 'min_child_weight': 12, 'n_estimators': 641}

SCORE: -553098.604 params {'colsample_bytree': 0.8, 'gamma': 6.782284556145175, 'lambda': 1.7287324913421842, 'learning_rate': 0.012823791237591541, 'max_depth': 21, 'min_child_weight': 42, 'n_estimators': 609}
SCORE: -549600.958 params {'colsample_bytree': 0.8, 'gamma': 6.13841794293382, 'lambda': 1.363396776977516, 'learning_rate': 0.023943715304410577, 'max_depth': 19, 'min_child_weight': 17, 'n_estimators': 614}
SCORE: -552026.812 params {'colsample_bytree': 0.8, 'gamma': 5.340472361867451, 'lambda': 1.750633857991943, 'learning_rate': 0.014708696075720352, 'max_depth': 18, 'min_child_weight': 27, 'n_estimators': 614}
SCORE: -547578.521 params {'colsample_bytree': 0.8, 'gamma': 6.418961485185021, 'lambda': 1.0558594532067118, 'learning_rate': 0.02810506941893589, 'max_depth': 21, 'min_child_weight': 28, 'n_estimators': 614}
SCORE: -549253.292 params {'colsample_bytree': 0.8, 'gamma': 5.0137587490395905, 'lambda': 1.1360453316012982, 'learning_rate': 0.02261104004959845, 'max_depth': 20, 'min_child_weight': 50, 'n_estimators': 634}
SCORE: -547537.375 params {'colsample_bytree': 0.8, 'gamma': 6.809845694575199, 'lambda': 1.5929642745470307, 'learning_rate': 0.025002042818406546, 'max_depth': 21, 'min_child_weight': 26, 'n_estimators': 633}
SCORE: -552058.958 params {'colsample_bytree': 0.8, 'gamma': 6.747949491761628, 'lambda': 1.7459168030014796, 'learning_rate': 0.024078009028246715, 'max_depth': 21, 'min_child_weight': 13, 'n_estimators': 606}
SCORE: -552472.562 params {'colsample_bytree': 0.8, 'gamma': 5.632946149962807, 'lambda': 1.1275490848575256, 'learning_rate': 0.01829268622400661, 'max_depth': 20, 'min_child_weight': 12, 'n_estimators': 641}
SCORE: -554191.271 params {'colsample_bytree': 0.8, 'gamma': 6.830382180591802, 'lambda': 1.9371200427919115, 'learning_rate': 0.023490551536353016, 'max_depth': 19, 'min_child_weight': 9, 'n_estimators': 614}
